In [ ]:
'''
Self RAG Implementation
Author: Christian Sarmiento
Purpose: This notebook is intended to get a Self-RAG implementation set up with LangChain/LangGraph.
Date Created: 11/17/24
Last Updated: 11/29/24
Data: Marist College Administrative Corpus Dataset
Sources:
- https://blog.langchain.dev/agentic-rag-with-langgraph/
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_self_rag.ipynb?ref=blog.langchain.dev
- ChatGPT: o1-preview
Note: Most of the code for graph implementation of Self-RAG was taken from the second source.
-----------------------------------------------------------------------------------------------------------------------
RAG Research             |               Machine Learning Independent Study             |              DR. EITEL LAURIA
'''

In [23]:
%pip install langgraph

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.6
    Uninstalling langchain-core-0.3.6:
      Successfully uninstalled langchain-core-0.3.6
Note: you may need to restart the kernel to use updated packages.


In [104]:
# Imports
import sys
sys.path.append("/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Private Code")
from api_keys import openAIKey
from api_keys import langchainKey
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import List, Dict, Optional, Annotated
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START


import pandas as pd
import os
import gradio as gr  # easy frontend implementation
from pprint import pprint
import numpy as np


from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import SingleTurnSample

In [2]:
# LangChain Enviornment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langchainKey()
os.environ["OPENAI_API_KEY"] = openAIKey()

In [3]:
# Load Data
csvPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/Marist_QA.csv"
maristQA = pd.read_csv(csvPath, header=None)

# To use RecursiveCharacterTextSplitter, we need a list of dictionaries
maristContext = [Document(page_content=text) for text in maristQA[1].tolist()]

In [4]:
# Split Documents into Chunks
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
texts = textSplitter.split_documents(maristContext)

In [5]:
# Store Documents in Vector DB (Chroma)
vectorDB = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

# Setup Retrieval System
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieves 3 documents

In [6]:
# Define Grader class for document grading in Self-RAG
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llmDocGrader = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMGrader = llmDocGrader.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
   tic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
gradePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Grader Chain
retrievalGrader = gradePrompt | structuredLLMGrader

# Testing it
question = "agent memory"
docs = retriever.get_relevant_documents(question)
docTxt = docs[1].page_content
print(retrievalGrader.invoke({"question": question, "document": docTxt}))

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_61612/3869534235.py:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='no'


In [7]:
# Generation Chain

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
ragChain = prompt | llm | StrOutputParser()

# Run
generation = ragChain.invoke({"context": docs, "question": question})
print(generation)

I don't know.


In [8]:
# Define Hallucination Grader Class
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llmHallucinations = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMHallucinationGrader = llmHallucinations.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucinationPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucinationGrader = hallucinationPrompt | structuredLLMHallucinationGrader
hallucinationGrader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='no')

In [9]:
# Answer Grader
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

# LLM with function call
llmAnswerGrading = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMAnswerGrader = llmAnswerGrading.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answerPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

# Chain
answerGrader = answerPrompt | structuredLLMAnswerGrader
answerGrader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

In [10]:
# Question Rewriter
llmRewriter = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
rewritePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

questionRewriter = rewritePrompt | llm | StrOutputParser()
questionRewriter.invoke({"question": question})

'What are the key concepts and functionalities of agent memory in artificial intelligence?'

In [11]:
# Define metric evaluator

## Evaluation LLM & embeddings
evalLLM = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evalEmbeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

## Initialize metrics with LLM and embeddings
contextRecall = LLMContextRecall(llm=evalLLM)
faithfulness = Faithfulness(llm=evalLLM)
factualCorrectness = FactualCorrectness(llm=evalLLM)
semanticSimilarity = SemanticSimilarity(embeddings=evalEmbeddings)

## Collect metrics
evalMetrics = [
    contextRecall,
    faithfulness,
    factualCorrectness,
    semanticSimilarity
]

In [113]:
# Define Graph structure for Self-RAG

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        metrics: evaluation metrics for each generation 
    """
    question: Annotated[str, "question_metadata"]
    generation: str
    documents: List[str]
    metrics: Optional[Dict[str, float]]
    retry_count: int = 0


## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Initialize retry count if not set
    print(f"Current Retry Count: {state["retry_count"]}")

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "retry_count": state["retry_count"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print(f"Current Retry Count: {state['retry_count']}")
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]


    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"Current Retry Count: {state['retry_count']}")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Score each doc
    filtered_docs = []
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"documents": filtered_docs, "question": question, "retry_count": retry_count + 1}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Retry Count: {retry_count}")
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Re-write question
    better_question = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question, "retry_count": retry_count + 1}


## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]
    retry_count = state["retry_count"] + 1

    # Recursion base case
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "generate"
    
    # All documents have been filtered check_relevance
    # We will re-generate a new query
    if not filtered_documents:
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    # We have relevant documents, so generate answer
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print(f"Current Retry Count: {state['retry_count']}")
    
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    score = hallucinationGrader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    
    # Recursion base case
    retry_count = state["retry_count"] + 1
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "useful"
    
    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")

        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answerGrader.invoke({"question": question, "generation": generation})
        grade = score.binary_score

        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    retry_count = state.get("retry_count", 0)
    print(f"Final Retry Count Before Reset: {retry_count}")
    state["retry_count"] = 0
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state

In [114]:
# Build Self-RAG Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery)  
workflow.add_node("evaluateMetrics", evaluateMetrics)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate"
    },
)
workflow.add_edge("transformQuery", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "evaluateMetrics",
        "not useful": "transformQuery",
    },
)
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("evaluateMetrics", END)


# Compile
app = workflow.compile()


In [ ]:
# Inital test run
inputs = {"question": "Who is Carolyn Matheus?"}
async for output in app.astream(inputs):
    for key, value in output.items():

        # Print node
        pprint(f"Node '{key}':")

        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)

        # Print metrics
        if "metrics" in value:
            pprint("Metrics: ")
            pprint(value["metrics"])

    pprint("\n---\n")

# Final generation
pprint("Final Generation: ")
pprint(value["generation"])

# Final metrics
if "metrics" in value:
    pprint("Final Metrics: ")
    pprint(value["metrics"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
'\n---\n'
'Final Generation: '
('Dr. Carolyn C. Matheus is an Associate Professor of Information Systems and '
 'the Director of the Honors Program at Marist College. She holds a PhD in '
 'Organizational Studies with a focus on leadership from SUNY Albany and has '
 'received the National Society of Leadership and Success award for Excellenc

In [115]:
# Put SelfRAG into Gradio
evaluationSamples = []
async def selfRAG(userQuery, history, correctAnswer):
    """
    Gradio-compatible function to process SelfRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        tuple: (chatDisplay, history)
    """

    # Get our input together
    inputs = {"question": userQuery, "retry_count": 0}

    # Start the workflow
    finalOutput = None
    async for output in app.astream(inputs, config={"recursion_limit": 100}):

        # Saving final output for metric purposes
        finalOutput = output

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Display output for gradio
    chatDisplay = [(msg["content"], "User" if msg["role"] == "user" else "LLM") for msg in history]

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })

    

    return history  #, chatDisplay

In [ ]:
# Gradio frontend
interface = gr.Interface(
    fn=selfRAG,
    inputs=[
        gr.Textbox(label="Ask a Question", placeholder="Enter your question here..."),
        gr.State(),  # Keeps track of conversation history
        gr.Textbox(label="Correct Answer (Optional)", placeholder="For evaluation purposes..."),
    ],
    outputs=[
        gr.Chatbot(label="SelfRAG Conversation"),
        gr.State(),  # Updates conversation history
    ],
    title="Self-RAG Implementation",
    description="Interact with the Self-RAG workflow for document-grounded question answering.",
)

# Launch the interface
interface.launch()

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTIO

In [116]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"input_query": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats
    

In [16]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [61]:
# Evaluate our pipeline responses
evalResults = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

{'input_query': 'Who is Carolyn Matheus?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.928909234587568}


In [118]:
# Sample 222 records from our dataset
maristTestSample = maristQA.sample(10, replace=False)
maristTestSample.head()

,0,1
159,Do most students at Marist go on a term abroad?,"""Study AbroadMarist AbroadAbout UsWith more th..."
291,Percentage of students completing Internships ...,"""InternshipsFind an InternshipIn recent years,..."
369,Where are computer science classes held?,"""Master of Science in Computer Science \u2013 ..."
470,Who is Irma Blanco Casey?,"""Contact InformationAcademic SchoolOfficeEmail..."
215,Is there a security escort service available t...,"""Safety and SecurityReport an IncidentTo repor..."


In [119]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    chatHistory = await selfRAG(row[1][0], chatHistory, row[1][1])
    
    
## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 2
---DECISION: GENERATE---
"Node 'gradeDocuments':"
Current Retry Count: 1
---GENERATE---
---CHECK HALLUCINATIONS---
Current Retry Count: 1
Current Retry Count: 2
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
Final Retry Count Before Reset: 1
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7170908372718965}
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMEN

CancelledError: 